# 🏗️ Structural Modeling Lab: E-commerce System

Welcome to this interactive lab! We will explore **Structural Modeling** concepts by building a simplified **E-commerce Engine** (like Shopify or Amazon) in Python.

### 🎯 Learning Objectives
1.  **Domain Classes**: How to represent "Things" (Products, Customers).
2.  **Associations**: How objects link to each other (Customer places Order).
3.  **Inheritance**: Handling special types (Physical vs Digital Products).
4.  **Composition**: Whole-Part relationships (Order contains Items).
5.  **State Machines**: Managing object life cycles (Order Status).

---

## 1. Domain Classes & Attributes

First, we define the core "Things" in our domain. In an E-commerce system, the most basic thing is a **Product**.

*   **Class**: `Product`
*   **Attributes** (What it knows): `sku`, `name`, `price`.
*   **Methods** (What it does): `display_info()`.

In [ ]:
class Product:
    def __init__(self, sku, name, price):
        self.sku = sku
        self.name = name
        self.price = price

    def display_info(self):
        return f"[{self.sku}] {self.name}: ${self.price}"

# Let's create some simple products
p1 = Product("SKU001", "Fancy T-Shirt", 25.00)
p2 = Product("SKU002", "Coffee Mug", 12.50)

print(p1.display_info())
print(p2.display_info())

## 2. Associations (Relationships)

Objects interact. A **Customer** *places* an **Order**.

*   **Multiplicity**: One Customer can have Many Orders (`1..*`).
*   **Direction**: The Order usually knows who placed it.

In [ ]:
class Customer:
    def __init__(self, customer_id, name):
        self.customer_id = customer_id
        self.name = name
        self.orders = []  # Association: Customer holds list of Orders (1..*)

    def place_order(self, order):
        self.orders.append(order)
        print(f"{self.name} placed a new order: {order.order_id}")

class Order:
    def __init__(self, order_id, customer):
        self.order_id = order_id
        self.customer = customer  # Association: Order linked to 1 Customer (1..1)

# Usage
alice = Customer(101, "Alice Wonderland")
order_1 = Order("ORD-2024-001", alice)

alice.place_order(order_1)

# Verify linkage
print(f"Order {order_1.order_id} belongs to: {order_1.customer.name}")

## 3. Generalization (Inheritance)

Not all products are the same. A **Digital Product** (like an eBook) has a download link, while a **Physical Product** has weight and shipping cost.

*   **Superclass**: `Product`
*   **Subclasses**: `DigitalProduct`, `PhysicalProduct`

In [ ]:
class DigitalProduct(Product):  # Inherits from Product
    def __init__(self, sku, name, price, download_url):
        super().__init__(sku, name, price)
        self.download_url = download_url

    def display_info(self):
        # Overriding parent method
        return f"[DIGITAL] {super().display_info()} -> Download: {self.download_url}"

class PhysicalProduct(Product):
    def __init__(self, sku, name, price, weight_kg):
        super().__init__(sku, name, price)
        self.weight_kg = weight_kg

    def display_info(self):
        return f"[PHYSICAL] {super().display_info()} (Weight: {self.weight_kg}kg)"

# Usage
ebook = DigitalProduct("D001", "Python Mastery PDF", 9.99, "http://site.com/dl")
dumbbells = PhysicalProduct("P001", "Gym Weights", 49.99, 10.5)

print(ebook.display_info())
print(dumbbells.display_info())

## 4. Composition (Whole-Part)

**Composition** is a strong relationship where the parts cannot exist without the whole.

*   **Comparison**: If you delete an **Order**, you must delete its **OrderItems** (line items like "2x Socks"). Those items make no sense without the order.
*   This is different from Aggregation (e.g., A Catalog has Products; if you delete Catalog, Products still exist).

In [ ]:
class OrderItem:
    def __init__(self, product, quantity):
        self.product = product
        self.quantity = quantity
    
    def get_total(self):
        return self.product.price * self.quantity

class CompositeOrder(Order): # Extending our previous Order class
    def __init__(self, order_id, customer):
        super().__init__(order_id, customer)
        self.items = [] # Composition: Items are created/managed INSIDE the Order

    def add_item(self, product, quantity):
        # The Order creates the OrderItem. 
        # The Item is tightly bound to this Order instance.
        new_item = OrderItem(product, quantity)
        self.items.append(new_item)
        print(f"Added {quantity}x {product.name} to Order {self.order_id}")

    def calculate_total(self):
        return sum(item.get_total() for item in self.items)

# Usage
my_order = CompositeOrder("ORD-999", alice)
my_order.add_item(ebook, 1)
my_order.add_item(dumbbells, 2)

print(f"Order Total: ${my_order.calculate_total():.2f}")

## 5. State Machine (Object Life Cycle)

Structural modeling also includes **State Machines**. Objects like Orders change their state over time (New -> Paid -> Shipped).

We use **Guard Conditions** to ensure valid transitions (e.g., can't Ship before Paying).

In [ ]:
class OrderStateMachine:
    # States
    NEW = "NEW"
    PAID = "PAID"
    SHIPPED = "SHIPPED"
    DELIVERED = "DELIVERED"
    CANCELLED = "CANCELLED"

    def __init__(self):
        self.state = self.NEW
        print(f"Order created. State: {self.state}")

    def pay(self):
        if self.state == self.NEW:
            self.state = self.PAID
            print("PAYMENT ACCEPTED. State -> PAID")
        else:
            print(f"Error: Cannot pay in state {self.state}")

    def ship(self):
        # Guard Condition: Must be PAID to ship
        if self.state == self.PAID:
            self.state = self.SHIPPED
            print("ITEM SHIPPED. State -> SHIPPED")
        elif self.state == self.NEW:
            print("Error: Customer hasn't paid yet!")
        else:
            print(f"Error: Cannot ship in state {self.state}")

    def cancel(self):
        if self.state in [self.NEW, self.PAID]:
            self.state = self.CANCELLED
            print("ORDER CANCELLED. State -> CANCELLED")
        else:
            print("Error: Too late to cancel!")

# Scenario 1: Happy Path
print("--- Scenario 1 ---")
o1 = OrderStateMachine()
o1.pay()
o1.ship()

# Scenario 2: Error Path (Shipping before Payment)
print("\n--- Scenario 2 ---")
o2 = OrderStateMachine()
o2.ship()

## 🏁 Conclusion

You have successfully modeled an E-commerce system using:
1.  **Classes** for domain objects.
2.  **Associations** to link Customers to Orders.
3.  **Inheritance** to specialize Product types.
4.  **Composition** to bind Items to Orders.
5.  **State Logic** to control Order workflows.